In [1]:
# import needed dependencies

#import yolo 
from ultralytics import YOLO

# import torch 
import torch
from torchvision import transforms

# import os and PIL for model evaluation 
import os
from PIL import Image

# import cv2 for image handling
import cv2


#import numpy
import numpy as np

#importing mqtt dependencies
import paho.mqtt.client as mqtt
import time


#chechking torch version
print(torch.__version__)

#setting manual seed
torch.manual_seed(42)

#clearing out the allocated storage used by the previous model
torch.cuda.empty_cache()



2.1.1+cu121


In [2]:
# EXPORTING MODEL INTO DIFFERENT FORMATS AND COMPARING THE RESULTS
#model.export(format = 'onnx')

In [3]:
# DEFUALT MODEL FORMAT 
model_path = os.path.join('.', 'runs', 'detect', 'train6', 'weights', 'best.pt')
model = YOLO(model_path)

In [4]:
# MODEL WITH NCNN FORMAT
model_ncnn_path = os.path.join('.', 'runs', 'detect', 'train6', 'weights','best.torchscript')
model_ncnn = YOLO(model_ncnn_path)

In [5]:
# MODEL WITH ONNX FORMAT 
model_onnx_path = os.path.join('.', 'runs', 'detect', 'train6', 'weights','best.onnx')
model_onxx = YOLO(model_onnx_path)

In [6]:
# create a client to send data to the plc
def start_connection_with_mqtt(client_id, MqttBroker):
    client  = mqtt.Client(mqtt.CallbackAPIVersion.VERSION1,client_id)
    client.connect(MqttBroker)
    return client

def publish_message(client, message, topic):
    client.publish(topic, message)
    print(f"Published: {message} to topic {topic}")
    #time.sleep(0.1)
node_red_client_new = start_connection_with_mqtt("bottle", "broker.hivemq.com")

C:\Users\omarm\AppData\Local\Temp\ipykernel_820\1890974757.py:3: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client  = mqtt.Client(mqtt.CallbackAPIVersion.VERSION1,client_id)


In [7]:
def send_plc_signal(client, class_id, topic):
    publish_message(client, class_id, topic)

In [16]:
def visualize_predictions(frame, predictions, half_frame_up, half_frame_down):

   # Get image dimensions
    height, width, _ = frame.shape
    # Draw bounding boxes and labels on the image
    for pred in predictions:

        # Extracxting needed data from model predictions
        class_id, confidence = pred.boxes.data.tolist()[0][5], pred.boxes.data.tolist()[0][4]
        class_name_dict = pred.names
        x_center, y_center, bbox_width, bbox_height = pred.boxes.xywhn.tolist()[0]
        if confidence > 0.7 :
        # some visualization to see what is happening
            down = (int(x_center * width), 0)
            up = (int(x_center * width), height)
            cv2.line(frame, pt1 = down, pt2 = up, color = (255, 255, 255), thickness=2)
            
            # Convert relative coordinates to absolute coordinates
            x1 = int((x_center - bbox_width / 2) * width)
            y1 = int((y_center - bbox_height / 2) * height)
            x2 = int((x_center + bbox_width / 2) * width)
            y2 = int((y_center + bbox_height / 2) * height)
            
            #checking weither the center point crosses the center of the camera frame or not 
            #if x_center * width == (width / 2):
            if x_center * width >= (width / 2) - 15 and x_center * width <= (width / 2) + 15 :
                cv2.line(img = frame, pt1= half_frame_up, pt2 = half_frame_down, color = (0, 0, 0), thickness = 3)
                # send 1 to the node-red and wait for acknowladgement then send the class id
                send_plc_signal(node_red_client_new, 1, "bottle_detected_new")
                send_plc_signal(node_red_client_new, class_id + 2, "bottle_color_new")
            

            # Draw bounding box
            cv2.rectangle(img = frame, pt1 = (x1, y1), pt2 = (x2, y2), color = (0, 255, 0), thickness = 2)
            
            # Write class label and confidence
            label = f'{class_name_dict[class_id]}: accu :{confidence:.2f}'
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2) 

        return frame

In [17]:
# Drwaing a line in the middle of the camera frame so we can notice if the process is going correctly or not
# Here we are just declaring the cordinates of the line 
half_frame_down = (320, 0)
half_frame_up = (320,480)


#choosing two differenct colors so if the bottle in the middle we change that color also to see if the process is going correctly
color1= (0, 255, 0)
color2 = (255, 0, 0)

In [18]:
cap = cv2.VideoCapture(0)
# to calculate the fps we need to calculate the starting time that that process starts and also when it ends
start_time = 0
# end time 
end_time = 0

while True :

    ret, frame = cap.read()

    if not ret :
        break
    
    # Doing forward path with inference mode so that the model does not tweak its perameters
    with torch.inference_mode():
        results = model_ncnn(frame)

    # checking if there are results ----> visualizting predictions
    if len(results[0].boxes.data) != 0 :
        frame = visualize_predictions(frame, results, half_frame_up, half_frame_down)
    
    # If there is no predictions then initialize the messages sent to node red with zero
    else :
        # One to stop convyer and the other to tell the bottle color
        send_plc_signal(node_red_client_new, 0, "bottle_detected_new")
        send_plc_signal(node_red_client_new, 0, "bottle_color_new")

    #Calculating the whole process time 
    end_time = time.time()

    # Calculating the fps
    fps = int(1/(end_time - start_time))
    fps2 = f"FPS : {fps}"
    start_time = end_time
    
    #writing the fps on the screen
    cv2.putText(frame, fps2, (10, 35),cv2.FONT_HERSHEY_SIMPLEX, 1.6, (0, 255, 0), 2)

    # opening the web cam with predictions
    cv2.imshow("predictions", frame)


    # if you press s then break the programm
    if cv2.waitKey(1) & 0xFF == ord('s'):
        break

# release all allocated memory 
cap.release()

# destroying the windows
cv2.destroyAllWindows() 


0: 640x640 1 blue bottle, 19.4ms
Speed: 0.0ms preprocess, 19.4ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 blue bottle, 13.9ms
Speed: 0.0ms preprocess, 13.9ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 blue bottle, 11.2ms
Speed: 2.4ms preprocess, 11.2ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 blue bottle, 12.0ms
Speed: 2.6ms preprocess, 12.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 blue bottle, 12.7ms
Speed: 0.0ms preprocess, 12.7ms inference, 3.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 blue bottle, 11.5ms
Speed: 0.0ms preprocess, 11.5ms inference, 5.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 blue bottle, 10.5ms
Speed: 2.9ms preprocess, 10.5ms inference, 4.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 blue bottle, 10.9ms
Speed: 1.7ms preprocess, 10.9ms inference, 3.9ms 

In [15]:
torch.cuda.empty_cache()